In [ ]:
# import open3d.ml.torch as ml3d
# or import open3d.ml.tf as ml3d
import numpy as np

# num_points = 100000
# points = np.random.rand(num_points, 3).astype(np.float32)

# # data = [
# #     {
# #         'name': 'my_point_cloud',
# #         'points': points,
# #         'random_colors': np.random.rand(*points.shape).astype(np.float32),
# #         'int_attr': (points[:,0]*5).astype(np.int32),
# #     }
# # ]

# # vis = ml3d.vis.Visualizer()
# # vis.visualize(data)

In [ ]:
# waymo_labels = ml3d.datasets.Waymo.get_label_to_names()
# waymo_labels

In [ ]:
import pickle 

In [ ]:
from datetime import datetime
import time

In [96]:
# datetime.now()-datetime.now()
s = time.time()
time.sleep(5)
elapsed = time.time() - s
print('Elapsed time is {} seconds.'.format(time.process_time()-time.process_time()))

Elapsed time is -1.0944999999651372e-05 seconds.


In [97]:
str(elapsed)

'5.001453161239624'

In [98]:
def rotate_point_cloud(data):
    """ Randomly rotate the point clouds to augument the dataset
        rotation is per shape based along up direction
        Input:
          Nx3 array, original point clouds
        Return:
          Nx3 array, rotated point clouds
    """
    rotated_data = np.zeros(data.shape, dtype=np.float32)

    rotation_angle = np.random.uniform() * 2 * np.pi
    cosval = np.cos(rotation_angle)
    sinval = np.sin(rotation_angle)
    rotation_matrix = np.array([[cosval, 0, sinval],
                                [0, 1, 0],
                                [-sinval, 0, cosval]])
    rotated_data = np.dot(data.reshape((-1, 3)), rotation_matrix)

    return rotated_data, rotation_matrix

In [99]:
with open("/home/jeevanandam/thesis/SuperGlue-pytorch/data/waymo/seq_0_frame_0.pkl", 'rb') as f:
        superglue = pickle.load(f)

In [100]:
pc1, R = rotate_point_cloud(superglue['points'][:, :3])

In [101]:
superglue['detections']['boxes'][:,:3].dtype

dtype('float32')

In [102]:
t = (R @ superglue['detections']['boxes'][:,:3].T).T

In [103]:
np.dot(superglue['detections']['boxes'][:,:3].reshape((-1, 3)), R).dtype

dtype('float64')

## GT Matches compute Load Data

In [27]:
with open("/home/jeevanandam/thesis/CenterPoint/tools/visualization/seq_0_frame_3_train_lidar.pkl", 'rb') as f:
        frame3_lidar = pickle.load(f)

In [28]:
with open("/home/jeevanandam/thesis/CenterPoint/tools/visualization/seq_0_frame_3_train_annos.pkl", 'rb') as f:
        frame3_annos = pickle.load(f)

In [29]:
with open("/home/jeevanandam/thesis/CenterPoint/tools/visualization/seq_0_frame_2_train_lidar.pkl", 'rb') as f:
        frame2_lidar = pickle.load(f)
with open("/home/jeevanandam/thesis/CenterPoint/tools/visualization/seq_0_frame_2_train_annos.pkl", 'rb') as f:
        frame2_annos = pickle.load(f)

In [30]:
first_frame = frame2_annos.copy()
second_frame = frame3_annos.copy()

In [31]:
first_frame_box_names = {}
for f in first_frame['objects']:
    first_frame_box_names[f['name']] = f['box']

second_frame_box_names = {}
for f in second_frame['objects']:
    second_frame_box_names[f['name']] = f['box']

In [32]:
kps1 = np.array(list(first_frame_box_names.values()))[:,:3]
kps2 = np.array(list(second_frame_box_names.values()))[:,:3]

In [37]:
matches_1_to_2 = np.nonzero(np.in1d(list(first_frame_box_names.keys()), list(second_frame_box_names.keys())))[0]
matches_2_to_1 = np.nonzero(np.in1d(list(second_frame_box_names.keys()), list(first_frame_box_names.keys())))[0]
matches_1_to_2

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70])

In [38]:
matches_2_to_1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71])

In [111]:
all_matches = np.vstack((matches_1_to_2, matches_2_to_1))
all_matches

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
        49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
        65, 66, 67, 68, 69, 70],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
        66, 67, 68, 69, 70, 71]])

In [112]:
scores_np1 = np.zeros(kps1.shape[0])
scores_np2 = np.zeros(kps2.shape[0])
scores_np1.shape, scores_np2.shape

((71,), (72,))

In [113]:
scores_np1[matches_1_to_2] = 1.0
scores_np2[matches_2_to_1] = 1.0

In [114]:
scores_np1.shape, scores_np2.shape

((71,), (72,))

In [115]:
frame2_annos

{'scene_name': '7741361323303179462_1230_310_1250_310',
 'frame_name': '7741361323303179462_1230_310_1250_310_location_phx_Day_1507675954833702',
 'frame_id': 2,
 'veh_to_global': array([ 1.09650308e-01, -9.93403153e-01,  3.35706033e-02,  1.99601531e+03,
         9.93969186e-01,  1.09538771e-01, -5.14935864e-03,  2.88844467e+03,
         1.43810650e-03,  3.39327739e-02,  9.99423083e-01,  2.89720000e+01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]),
 'objects': [{'id': 0,
   'name': '0K_z0NgwZCS8kesbfkmBcg',
   'label': 1,
   'box': array([ 2.2089382e+01, -3.7525234e+01,  2.2267904e+00,  5.1498923e+00,
           2.2411187e+00,  1.7300000e+00, -1.4737081e-03, -3.1719714e-02,
          -1.4874748e+00], dtype=float32),
   'num_points': 138,
   'detection_difficulty_level': 0,
   'combined_difficulty_level': 1,
   'global_speed': array([ 0.0313851 , -0.00494492], dtype=float32),
   'global_accel': array([ 1.1121627e-13, -7.5063992e-14], dtype=float32)},
  {'

In [116]:
with open("/home/jeevanandam/thesis/CenterPoint/tools/visualization/infos_train_01sweeps_filter_zero_gt.pkl", 'rb') as f:
        infos = pickle.load(f)

In [117]:
proj_frame2 = frame2_annos['veh_to_global'].reshape(4,4)
proj_frame3 = frame3_annos['veh_to_global'].reshape(4,4)

In [118]:
rotation_frame2 = frame2_annos['veh_to_global'].reshape(4,4)[:3, :3] 
trans_frame_2 = frame2_annos['veh_to_global'].reshape(4,4)[:3, 3]

In [119]:
rotation_frame3 = frame3_annos['veh_to_global'].reshape(4,4)[:3, :3] 
trans_frame_3 = frame3_annos['veh_to_global'].reshape(4,4)[:3, 3]

In [120]:
rel_rot = rotation_frame3.T @ rotation_frame2
rel_trans = rotation_frame3.T @ (trans_frame_2 - trans_frame_3)

In [121]:
box1_frame2 = frame2_annos['objects'][0]['box'][:3]
box1_frame3 = frame3_annos['objects'][0]['box'][:3]


In [122]:
# box1_frame2 = np.insert(box1_frame2, 3, 1, axis=0)
# box1_frame2

In [123]:
box1_frame2_in_frame3 = rel_rot @ box1_frame2 + rel_trans

In [124]:
box1_frame2_in_world = rotation_frame2 @ box1_frame2 + trans_frame_2
box1_frame3_in_world = rotation_frame3 @ box1_frame3 + trans_frame_3

box1_frame2_in_world_to_frame3 = rotation_frame3.T @ (box1_frame2_in_world - trans_frame_3)

In [125]:
print(box1_frame2_in_world)
print(box1_frame3_in_world)

[2035.78985749 2906.27889581   29.95593735]
[2035.79299716 2906.27840007   29.95569951]


In [126]:
print(box1_frame3)
print(box1_frame2_in_frame3)
print(box1_frame2_in_world_to_frame3)

[ 21.253815  -37.430252    2.3023436]
[ 21.25394932 -37.42707      2.30247059]
[ 21.25394932 -37.42707      2.30247059]


In [127]:
print(box1_frame3)
print(box1_frame2_in_frame3)

[ 21.253815  -37.430252    2.3023436]
[ 21.25394932 -37.42707      2.30247059]


In [128]:
infos[0]

{'path': 'data/Waymo/train/lidar/seq_0_frame_0.pkl',
 'anno_path': 'data/Waymo/train/annos/seq_0_frame_0.pkl',
 'token': 'seq_0_frame_0.pkl',
 'timestamp': 1507265171.807778,
 'sweeps': [],
 'gt_boxes': array([[-1.42703819e+01,  6.48398819e+01, -4.08101141e-01,
          1.81581271e+00,  3.83745241e+00,  1.63999999e+00,
          1.51626451e-03,  2.07198830e-03, -3.64491320e+00],
        [-1.71704197e+01,  4.56558304e+01,  1.22630931e-01,
          2.03115702e+00,  4.33986521e+00,  1.75999999e+00,
          7.09603028e-03,  3.47832800e-04, -3.67502213e+00],
        [ 1.11894846e+01, -5.84508286e+01,  2.33811927e+00,
          2.03578615e+00,  4.44594336e+00,  1.37000000e+00,
          2.71663442e-02,  6.23191707e-03,  9.10043716e-03],
        [ 1.15470457e+00,  6.32880554e+01, -3.47450495e-01,
          1.01241136e+00,  9.77332592e-01,  1.75999999e+00,
         -3.40635590e-02,  6.06974177e-02, -3.06618571e+00],
        [-8.64962292e+00,  4.66797371e+01,  1.50193619e-02,
          2.22

In [129]:
frame2_annos

{'scene_name': '7741361323303179462_1230_310_1250_310',
 'frame_name': '7741361323303179462_1230_310_1250_310_location_phx_Day_1507675954833702',
 'frame_id': 2,
 'veh_to_global': array([ 1.09650308e-01, -9.93403153e-01,  3.35706033e-02,  1.99601531e+03,
         9.93969186e-01,  1.09538771e-01, -5.14935864e-03,  2.88844467e+03,
         1.43810650e-03,  3.39327739e-02,  9.99423083e-01,  2.89720000e+01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]),
 'objects': [{'id': 0,
   'name': '0K_z0NgwZCS8kesbfkmBcg',
   'label': 1,
   'box': array([ 2.2089382e+01, -3.7525234e+01,  2.2267904e+00,  5.1498923e+00,
           2.2411187e+00,  1.7300000e+00, -1.4737081e-03, -3.1719714e-02,
          -1.4874748e+00], dtype=float32),
   'num_points': 138,
   'detection_difficulty_level': 0,
   'combined_difficulty_level': 1,
   'global_speed': array([ 0.0313851 , -0.00494492], dtype=float32),
   'global_accel': array([ 1.1121627e-13, -7.5063992e-14], dtype=float32)},
  {'

In [130]:
def sort_frames(frames):
    indices = [] 

    for f in frames:
        seq_id = int(f.split("_")[1])
        frame_id= int(f.split("_")[3][:-4])

        idx = seq_id * 1000 + frame_id
        indices.append(idx)

    rank = list(np.argsort(np.array(indices)))

    frames = [frames[r] for r in rank]
    return frames

In [131]:
import os

path = "/home/jeevanandam/thesis/CenterPoint/data/val/annos1"
pairs = []

files = []
seq_frames = {}


files += [f for f in sort_frames(os.listdir(path))]
for f in files:
    frame_num = '_'.join(f[:-4].split('_')[2:4])
    seq_name = '_'.join(f[:-4].split('_')[:2])
    if seq_name in seq_frames:
        seq_frames[seq_name].append(frame_num)
    else:
        seq_frames[seq_name] = [frame_num]
for seq, frames in seq_frames.items():
    next_frame_idx = 1
    num_of_frames = len(frames)
    for current_frame_idx in range(num_of_frames):
        if next_frame_idx == num_of_frames:
            break
        pairs.append((path + seq + frames[current_frame_idx], path + seq + frames[next_frame_idx]))
        next_frame_idx += 1
pairs

FileNotFoundError: [Errno 2] No such file or directory: '/home/jeevanandam/thesis/CenterPoint/data/val/annos1'

## CENTER POINT FEATURES

In [ ]:
import torch

In [ ]:
cp_model = torch.load("/home/jeevanandam/thesis/CenterPoint/work_dirs/waymo_centerpoint_pp_two_pfn_stride1_3x/epoch_36.pth")

In [ ]:
cp_model['state_dict'].keys()

odict_keys(['reader.pfn_layers.0.linear.weight', 'reader.pfn_layers.0.norm.weight', 'reader.pfn_layers.0.norm.bias', 'reader.pfn_layers.0.norm.running_mean', 'reader.pfn_layers.0.norm.running_var', 'reader.pfn_layers.0.norm.num_batches_tracked', 'reader.pfn_layers.1.linear.weight', 'reader.pfn_layers.1.norm.weight', 'reader.pfn_layers.1.norm.bias', 'reader.pfn_layers.1.norm.running_mean', 'reader.pfn_layers.1.norm.running_var', 'reader.pfn_layers.1.norm.num_batches_tracked', 'neck.blocks.0.1.weight', 'neck.blocks.0.2.weight', 'neck.blocks.0.2.bias', 'neck.blocks.0.2.running_mean', 'neck.blocks.0.2.running_var', 'neck.blocks.0.2.num_batches_tracked', 'neck.blocks.0.4.weight', 'neck.blocks.0.5.weight', 'neck.blocks.0.5.bias', 'neck.blocks.0.5.running_mean', 'neck.blocks.0.5.running_var', 'neck.blocks.0.5.num_batches_tracked', 'neck.blocks.0.7.weight', 'neck.blocks.0.8.weight', 'neck.blocks.0.8.bias', 'neck.blocks.0.8.running_mean', 'neck.blocks.0.8.running_var', 'neck.blocks.0.8.num_batc

In [ ]:
cp_model['state_dict']['bbox_head.tasks.0.hm.1.weight'].shape

torch.Size([64])

In [1]:
# modified from the single_inference.py by @muzi2045
from spconv.utils import VoxelGenerator as VoxelGenerator
from det3d.datasets.pipelines.loading import read_single_waymo
from det3d.datasets.pipelines.loading import get_obj
from det3d.torchie.trainer import load_checkpoint
from det3d.models import build_detector
from det3d.torchie import Config
from tqdm import tqdm 
import numpy as np
import pickle 
import open3d as o3d
import argparse
import torch
import time 
import os
from scipy.spatial.distance import cdist

from det3d.torchie.apis import get_root_logger
from det3d.torchie import Config

from det3d.core.bbox import box_np_ops

voxel_generator = None 
model = None 
device = None 
activation = {}

def initialize_model(config, checkpoint, fp16=False):
    global model, voxel_generator    
    cfg = Config.fromfile(config)
    model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
    if checkpoint is not None:
        load_checkpoint(model, checkpoint, map_location="cpu")
    # print(model)
    if fp16:
        print("cast model to fp16")
        model = model.half()

    model = model.cuda()
    model.eval()

    global device 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    range = cfg.voxel_generator.range
    voxel_size = cfg.voxel_generator.voxel_size
    max_points_in_voxel = cfg.voxel_generator.max_points_in_voxel
    max_voxel_num = cfg.voxel_generator.max_voxel_num[1]
    voxel_generator = VoxelGenerator(
        voxel_size=voxel_size,
        point_cloud_range=range,
        max_num_points=max_points_in_voxel,
        max_voxels=max_voxel_num
    )
    return model 



def sort_frames(frames):
    indices = [] 

    for f in frames:
        seq_id = int(f.split("_")[1])
        frame_id= int(f.split("_")[3][:-4])

        idx = seq_id * 1000 + frame_id
        indices.append(idx)

    rank = list(np.argsort(np.array(indices)))

    frames = [frames[r] for r in rank]
    return frames



no apex
No Tensorflow
Deformable Convolution not built!


In [2]:


# parser = argparse.ArgumentParser(description="CenterPoint")
# parser.add_argument("config", help="path to config file", default="/home/jeevanandam/thesis/CenterPoint_new/configs/waymo/pp/waymo_centerpoint_pp_two_pfn_stride1_3x.py", type=str)
# parser.add_argument(
#     "--checkpoint", help="the path to checkpoint which the model read from", default="/home/jeevanandam/thesis/CenterPoint_new/work_dirs/waymo_centerpoint_pp_two_pfn_stride1_3x/epoch_36.pth", type=str
# )
# # parser.add_argument('--input_data_dir', type=str, default="/home/jeevanandam/thesis/CenterPoint_new/data/train/lidar")
# # parser.add_argument('--annos_data_dir', type=str, default="/home/jeevanandam/thesis/CenterPoint_new/data/train/annos")
# # parser.add_argument('--output_dir', type=str, required=True)
# # parser.add_argument('--fp16', action='store_true')
# # parser.add_argument('--threshold', default=0.5)
# # parser.add_argument('--visual', action='store_true')
# # parser.add_argument("--online", action='store_true')
# # parser.add_argument('--num_frame', default=-1, type=int)
# args = parser.parse_args()

config = "/home/jeevanandam/thesis/CenterPoint_new/configs/waymo/pp/waymo_centerpoint_pp_two_pfn_stride1_3x.py"
checkpoint = "/home/jeevanandam/thesis/CenterPoint_new/work_dirs/waymo_centerpoint_pp_two_pfn_stride1_3x/epoch_36.pth"
# print("Please prepare your point cloud in waymo format and save it as a pickle dict with points key into the {}".format(args.input_data_dir))
print("One point cloud should be saved in one pickle file.")
print("Download and save the pretrained model at {}".format(checkpoint))

# Run any user-specified initialization code for their submission.
model = initialize_model(config, checkpoint)


One point cloud should be saved in one pickle file.
Download and save the pretrained model at /home/jeevanandam/thesis/CenterPoint_new/work_dirs/waymo_centerpoint_pp_two_pfn_stride1_3x/epoch_36.pth
Use HM Bias:  -2.19


In [3]:
def voxelization(points, voxel_generator):
    voxel_output = voxel_generator.generate(points)  
    voxels, coords, num_points = \
        voxel_output['voxels'], voxel_output['coordinates'], voxel_output['num_points_per_voxel']

    return voxels, coords, num_points  

def _process_inputs(points, fp16):
    voxels, coords, num_points = voxel_generator.generate(points)
    num_voxels = np.array([voxels.shape[0]], dtype=np.int32)
    grid_size = voxel_generator.grid_size
    coords = np.pad(coords, ((0, 0), (1, 0)), mode='constant', constant_values = 0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    voxels = torch.tensor(voxels, dtype=torch.float32, device=device)
    coords = torch.tensor(coords, dtype=torch.int32, device=device)
    num_points = torch.tensor(num_points, dtype=torch.int32, device=device)
    num_voxels = torch.tensor(num_voxels, dtype=torch.int32, device=device)

    if fp16:
        voxels = voxels.half()

    inputs = dict(
            voxels = voxels,
            num_points = num_points,
            num_voxels = num_voxels,
            coordinates = coords,
            shape = [grid_size]
        )

    return inputs 

def run_model(points, fp16=False):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    # model.bbox_head.tasks[0].reg[1].register_forward_hook(get_activation('reg'))
    model.reader.register_forward_hook(get_activation('reader'))
    # model.backbone.register_forward_hook(get_activation('backbone'))
    # model.neck.register_forward_hook(get_activation('neck'))
    # model.bbox_head.shared_conv.register_forward_hook(get_activation('before_reg'))
    with torch.no_grad():
        data_dict = _process_inputs(points, fp16)
        outputs = model(data_dict, return_loss=False)[0]

    # print(activation['reader'])
    return {'boxes': outputs['box3d_lidar'].cpu().numpy(),
        'scores': outputs['scores'].cpu().numpy(),
        'classes': outputs['label_preds'].cpu().numpy()}, data_dict, activation['reader']

def process_example(points, fp16=False):
    output, data_dict, inter_features = run_model(points, fp16)

    assert len(output) == 3
    assert set(output.keys()) == set(('boxes', 'scores', 'classes'))
    num_objs = output['boxes'].shape[0]
    assert output['scores'].shape[0] == num_objs
    assert output['classes'].shape[0] == num_objs

    return output, data_dict, inter_features


In [19]:
points1 = read_single_waymo(get_obj("/home/jeevanandam/thesis/CenterPoint_new/data/test/lidar/seq_0_frame_0.pkl"))
points2 = read_single_waymo(get_obj("/home/jeevanandam/thesis/CenterPoint_new/data/test/lidar/seq_0_frame_1.pkl"))

detections1, data_dicts1, inter_features1 = process_example(points1, False)
detections2, data_dicts2, inter_features2 = process_example(points2, False)

In [20]:
kps1 = detections1['boxes'][:,:3]
kps2 = detections2['boxes'][:,:3]

In [6]:
scores1 = detections1['scores']
scores2 = detections2['scores']

In [7]:
voxels1 = data_dicts1['voxels'].reshape(-1,5).cpu().numpy()
voxels_flattened1 = voxels1.reshape(-1,5)[:,:3]
keypoint_features1 = []
for keypoint in kps1:
    idx = cdist(voxels_flattened1, np.atleast_2d(keypoint)).argmin()
    voxel_id = int(np.floor(idx/20))
    keypoint_features1.append(inter_features1[voxel_id].cpu().numpy())
keypoint_features1 = np.array(keypoint_features1)

In [8]:
voxels2 = data_dicts2['voxels'].reshape(-1,5).cpu().numpy()
voxels_flattened2 = voxels2.reshape(-1,5)[:,:3]
keypoint_features2 = []
for keypoint in kps2:
    idx = cdist(voxels_flattened2, np.atleast_2d(keypoint)).argmin()
    voxel_id = int(np.floor(idx/20))
    keypoint_features2.append(inter_features2[voxel_id].cpu().numpy())
keypoint_features2 = np.array(keypoint_features2)

In [9]:
kps1 = kps1.reshape((1, -1, 3))
kps2 = kps2.reshape((1, -1, 3))

keypoint_features1 = keypoint_features1.T
keypoint_features2 = keypoint_features2.T

In [10]:
kps1.shape, kps2.shape, keypoint_features1.shape, keypoint_features2.shape

((1, 63, 3), (1, 59, 3), (64, 63), (64, 59))

In [11]:
torch.stack([torch.Tensor(x) for x in kps1]).shape

torch.Size([1, 63, 3])

In [22]:
input_superglue = {}
input_superglue["keypoints0"] = list(kps1)
input_superglue["keypoints1"] = list(kps2)
input_superglue["scores0"] = list(scores1)
input_superglue["scores1"] = list(scores2)
input_superglue["descriptors0"] = list(keypoint_features1)
input_superglue["descriptors1"] = list(keypoint_features2)

# input_superglue["keypoints0"] = list(torch.from_numpy(kps1))
# input_superglue["keypoints1"] = list(torch.from_numpy(kps2))
# input_superglue["scores0"] = list(torch.from_numpy(scores1))
# input_superglue["scores1"] = list(torch.from_numpy(scores2))
# input_superglue["descriptors0"] = [torch.Tensor(x) for x in keypoint_features1]
# input_superglue["descriptors1"] = [torch.Tensor(x) for x in keypoint_features1]



In [23]:
torch.from_numpy(kps1).shape

torch.Size([63, 3])

In [24]:
torch.save(input_superglue, "/home/jeevanandam/thesis/SuperGlue-pytorch/test_input.pkl")

In [ ]:
# keypoints0 torch.Size([1, 21, 3])
# keypoints1 torch.Size([1, 21, 3])
# descriptors0 torch.Size([1, 30])
# descriptors1 torch.Size([1, 30])
# scores0 torch.Size([1])
# scores1 torch.Size([1])
# all_matches torch.Size([1, 21])





# keypoints0        torch.Size([1, 1, 21, 3])
# keypoints1        torch.Size([1, 1, 21, 3])
# descriptors0      torch.Size([64, 1, 30])
# descriptors1      torch.Size([64, 1, 30])
# scores0           torch.Size([21, 1])
# scores1           torch.Size([21, 1])
# all_matches       torch.Size([2, 1, 21])


In [ ]:
# keypoints0 torch.Size([1, 63, 3])
# keypoints1 torch.Size([1, 59, 3])
# scores0 torch.Size([63])
# scores1 torch.Size([59])
# descriptors0 torch.Size([64, 63])
# descriptors1 torch.Size([64, 59])


In [41]:
with open("/home/jeevanandam/thesis/local_results/seq_3", 'rb') as f:
        infos = pickle.load(f)

In [45]:
infos['frame_0']['detections']['boxes']

array([[-23.585232  ,  -5.272278  ,   1.1983751 , ...,   8.488486  ,
          3.0747645 ,  -1.513756  ],
       [-34.01527   ,   8.824287  ,   0.13498396, ...,   4.6391587 ,
          1.4896554 ,   1.5329534 ],
       [ -2.9514008 ,   8.88903   ,   0.53215355, ...,   4.45161   ,
          1.7758027 ,   1.5558923 ],
       ...,
       [-16.533367  ,  31.83136   ,   0.31001413, ...,   5.6531196 ,
          2.1166008 ,  -1.874625  ],
       [ 37.856857  ,  23.470932  ,   0.70307493, ...,   0.8218262 ,
          1.7598282 ,  -1.3167558 ],
       [-33.089046  , -32.783257  ,   1.8474207 , ...,   4.5747485 ,
          1.661983  ,   0.20856293]], dtype=float32)

In [46]:
795/3

265.0

# GRAPHS FOR METRICS

In [113]:
import bokeh
import os
from bokeh.plotting import figure, output_notebook, show


output_notebook()

Loading BokehJS ...

In [76]:
root_dir = './work_dirs_cluster_mini/new/'
mAPs = []
mAPHs = []
exp_names = []
metrics_data = {}

In [107]:
'_'.join(os.listdir(root_dir)[3].split('_')[10:])

'gnn_fh_1920_to_256_6l_kenc_only'

In [108]:
obj_types = ['VEHICLE_LEVEL_1',
   'VEHICLE_LEVEL_2',
   'PEDESTRIAN_LEVEL_1',
   'PEDESTRIAN_LEVEL_2',
   'CYCLIST_LEVEL_1',
   'CYCLIST_LEVEL_2']

metrics_data = {}
for t in obj_types:
   metrics_data[t] = {
      'mAPs' : [],
      'mAPHs' : [],
      'exp_names' : [],
   }

In [109]:
metrics_data

{'VEHICLE_LEVEL_1': {'mAPs': [], 'mAPHs': [], 'exp_names': []},
 'VEHICLE_LEVEL_2': {'mAPs': [], 'mAPHs': [], 'exp_names': []},
 'PEDESTRIAN_LEVEL_1': {'mAPs': [], 'mAPHs': [], 'exp_names': []},
 'PEDESTRIAN_LEVEL_2': {'mAPs': [], 'mAPHs': [], 'exp_names': []},
 'CYCLIST_LEVEL_1': {'mAPs': [], 'mAPHs': [], 'exp_names': []},
 'CYCLIST_LEVEL_2': {'mAPs': [], 'mAPHs': [], 'exp_names': []}}

In [237]:
obj_types = ['VEHICLE_LEVEL_1',
   'VEHICLE_LEVEL_2',
   'PEDESTRIAN_LEVEL_1',
   'PEDESTRIAN_LEVEL_2',
   'CYCLIST_LEVEL_1',
   'CYCLIST_LEVEL_2']

type_ranges = dict(
    VEHICLE_LEVEL_1 = (60,80),
    VEHICLE_LEVEL_2 = (50,80),
    PEDESTRIAN_LEVEL_1 = (60,80),
    PEDESTRIAN_LEVEL_2 = (50,80),
    CYCLIST_LEVEL_1 = (45, 65),
    CYCLIST_LEVEL_2 = (45, 60),
)

metrics_data = {}
for t in obj_types:
   metrics_data[t] = {
      'mAPs' : np.array([]),
      'mAPHs' : np.array([]),
      'exp_names' : np.array([]),
   }

for exp_dir in os.listdir(root_dir):
    if not os.path.exists(root_dir + exp_dir + '/results.txt'):
        continue
    
    with open(root_dir + exp_dir + '/results.txt') as f:
        lines = f.readlines()
        for line in lines:
            if (line[:6] == "OBJECT") and (line.split(':')[0].split('_')[-3] != "SIGN"):
                obj_type = '_'.join(line.split(':')[0].split('_')[-3:])
                metrics_data[obj_type]['mAPs'] = np.append(metrics_data[obj_type]['mAPs'], (float(line.split(' ')[2][:-1])*100.0))
                metrics_data[obj_type]['mAPHs'] = np.append(metrics_data[obj_type]['mAPHs'], (float(line.split(' ')[4].strip()[:-1])*100.0))
                metrics_data[obj_type]['exp_names'] = np.append(metrics_data[obj_type]['exp_names'], '_'.join(exp_dir.split('_')[10:]))
                
# for t in obj_types:
#     mAPs_inds = metrics_data[t]['mAPs'].argsort()
#     metrics_data[t]['mAPs'] = metrics_data[t]['mAPs'][mAPs_inds[::-1]]
#     metrics_data[t]['exp_names_mAPs'] = metrics_data[t]['exp_names'][mAPs_inds[::-1]]
    
#     mAPHs_inds = metrics_data[t]['mAPHs'].argsort()
#     metrics_data[t]['mAPHs'] = metrics_data[t]['mAPHs'][mAPHs_inds[::-1]]
#     metrics_data[t]['exp_names_mAPHs'] = metrics_data[t]['exp_names'][mAPHs_inds[::-1]]
    

In [238]:

obj_types = list(metrics_data.keys())
for t in obj_types:
    print(metrics_data[t]['mAPs'])

[70.5691 71.7851 70.3174 71.6966 69.802  70.3846 69.1527 69.5935 71.6241]
[66.7906 67.989  66.5629 67.8968 66.0912 66.6669 65.5322 65.8763 67.7987]
[71.6523 74.3093 71.6577 73.4327 71.6966 71.834  72.0731 71.4346 72.4521]
[67.1642 69.7931 67.1822 68.8853 67.34   67.5593 67.8289 67.0778 67.9185]
[53.1691 53.5765 52.8836 55.0359 53.9405 50.9171 53.3477 52.3511 50.7923]
[50.5307 50.9478 50.2834 52.3254 51.2939 48.423  50.7482 49.7914 48.2887]


In [265]:
from bokeh.models import ColumnDataSource, LabelSet, DatetimeTickFormatter, Legend,Whisker, HoverTool
from bokeh.transform import dodge
from bokeh.models.tickers import MonthsTicker
from bokeh.layouts import gridplot
import numpy as np
from bokeh.io import curdoc



# y1 = [10 - i for i in x]
# y2 = [abs(i - 5) for i in x]
plots = []

for category in metrics_data.keys():
    source = ColumnDataSource(data=metrics_data[category])
    
    p = figure(x_range=metrics_data[category]['exp_names'], y_range=type_ranges[category])
    
    mAPs_line_glyph = p.line('exp_names', 'mAPs', line_width=2, source=source, line_color='#009E73', legend_label="mAP")
    mAPs_circle_glyph = p.circle('exp_names', 'mAPs', size=6, alpha=0.8, source=source)
    
    mAPHs_line_glyph = p.line('exp_names', 'mAPHs', line_width=2, source=source, line_color='#F0E442', legend_label="mAPH")
    mAPHs_circle_glyph = p.circle('exp_names', 'mAPHs', size=6, alpha=0.8, source=source)
    
    
    # Add the glyphs to the plot using the renderers attribute
    p.renderers.append(mAPs_line_glyph)
    p.renderers.append(mAPs_circle_glyph)
    p.renderers.append(mAPHs_line_glyph)
    p.renderers.append(mAPHs_circle_glyph)
    
    
    # Hover tooltip for flight lines, assign only the line renderer
    hover_mAP = HoverTool(tooltips=[('mAP', '@mAPs')],
                            renderers = [mAPs_circle_glyph])
    
    # Hover tooltip for origin and destination, assign only the line renderer
    hover_mAPH = HoverTool(tooltips=[('mAPH', '@mAPHs')],
                            renderers = [mAPHs_circle_glyph])

    # Add the hovertools to the figure
    p.add_tools(hover_mAP)
    p.add_tools(hover_mAPH)
    
    
    # style and others
    
    p.title = category
    p.xaxis.major_label_orientation = np.pi/4 
    # change just some things about the x-grid
    p.xgrid.grid_line_color = None

    # change just some things about the y-grid
    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.1

    plots.append(p)
# s2 = figure(background_fill_color="#fafafa")
# # s2.triangle(x, y1, size=12, alpha=0.8, color="#c02942")

# s3 = figure(background_fill_color="#fafafa")
# # s3.square(x, y2, size=12, alpha=0.8, color="#d95b43")

# make a grid
grid = gridplot(plots, ncols=2, width=500, height=500)

curdoc().theme = 'dark_minimal'
show(grid)


In [261]:
with open('results.txt') as f:
        lines = f.readlines()
for line in lines:
    if (line[:5] == "RANGE") and (line.split(':')[0].split('_')[-4] != "SIGN") and (line.split(':')[0].split('_')[-1] == "1"):
        # obj_type = (float(line.split(' ')[2][:-1])*100.0)
        # print(obj_type)
        ll = line
        break
ll.split(' ')[5].strip()[:-1]

'0.908803'

In [263]:
range_obj_types = [
    'VEHICLE_[0, 30)',
    'VEHICLE_[30, 50)',
    'VEHICLE_[50, +inf)',
    'PEDESTRIAN_[0, 30)',
    'PEDESTRIAN_[30, 50)',
    'PEDESTRIAN_[50, +inf)',
    'CYCLIST_[0, 30)',
    'CYCLIST_[30, 50)',
    'CYCLIST_[50, +inf)',
    ]

type_ranges = dict(
    VEHICLE_LEVEL_1 = (60,80),
    VEHICLE_LEVEL_2 = (50,80),
    PEDESTRIAN_LEVEL_1 = (60,80),
    PEDESTRIAN_LEVEL_2 = (50,80),
    CYCLIST_LEVEL_1 = (45, 65),
    CYCLIST_LEVEL_2 = (45, 60),
)

range_metrics_data = {}
for t in range_obj_types:
   range_metrics_data[t] = {
      'mAPs' : np.array([]),
      'mAPHs' : np.array([]),
      'exp_names' : np.array([]),
   }

for exp_dir in os.listdir(root_dir):
    if not os.path.exists(root_dir + exp_dir + '/results.txt'):
        continue
    
    with open(root_dir + exp_dir + '/results.txt') as f:
        lines = f.readlines()
        for line in lines:
            if (line[:5] == "RANGE") and (line.split(':')[0].split('_')[-4] != "SIGN") and (line.split(':')[0].split('_')[-1] == "1"):
                obj_type = '_'.join(line.split(':')[0].split('_')[-4:-2])
                range_metrics_data[obj_type]['mAPs'] = np.append(range_metrics_data[obj_type]['mAPs'], (float(line.split(' ')[3][:-1])*100.0))
                range_metrics_data[obj_type]['mAPHs'] = np.append(range_metrics_data[obj_type]['mAPHs'], (float(line.split(' ')[5].strip()[:-1])*100.0))
                range_metrics_data[obj_type]['exp_names'] = np.append(range_metrics_data[obj_type]['exp_names'], '_'.join(exp_dir.split('_')[10:]))
                
# for t in range_obj_types:
#     mAPs_inds = range_metrics_data[t]['mAPs'].argsort()
#     range_metrics_data[t]['mAPs'] = range_metrics_data[t]['mAPs'][mAPs_inds[::-1]]
#     range_metrics_data[t]['exp_names_mAPs'] = range_metrics_data[t]['exp_names'][mAPs_inds[::-1]]
    
#     mAPHs_inds = range_metrics_data[t]['mAPHs'].argsort()
#     range_metrics_data[t]['mAPHs'] = range_metrics_data[t]['mAPHs'][mAPHs_inds[::-1]]
#     range_metrics_data[t]['exp_names_mAPHs'] = range_metrics_data[t]['exp_names'][mAPHs_inds[::-1]]

In [264]:
range_metrics_data

{'VEHICLE_[0, 30)': {'mAPs': array([91.0237, 91.4739, 91.1688, 91.2535, 90.9209, 90.7294, 90.719 ,
         90.8545, 91.3752]),
  'mAPHs': array([90.5202, 90.9769, 90.6626, 90.7567, 90.3982, 90.1914, 90.1962,
         90.3373, 90.8803]),
  'exp_names': array(['max_combine', 'gnn_fh_1920_to_256_6l_kenc_only_d_64',
         'mean_combine', 'gnn_fh_1920_to_256_6l_kenc_only',
         'max_min_combine', 'gnn_2l_kenc_64_d_384', 'gnn_3l_nokenc_d_1920',
         'fh_max_min_combine', 'baseline'], dtype='<U36')},
 'VEHICLE_[30, 50)': {'mAPs': array([68.5661, 69.7655, 68.081 , 70.1282, 67.6968, 68.0689, 67.4566,
         67.4336, 69.8993]),
  'mAPHs': array([67.9682, 69.1055, 67.4966, 69.4671, 67.0833, 67.338 , 66.7853,
         66.8021, 69.2682]),
  'exp_names': array(['max_combine', 'gnn_fh_1920_to_256_6l_kenc_only_d_64',
         'mean_combine', 'gnn_fh_1920_to_256_6l_kenc_only',
         'max_min_combine', 'gnn_2l_kenc_64_d_384', 'gnn_3l_nokenc_d_1920',
         'fh_max_min_combine', 'basel